In [12]:
%pip install datasets ragas llama_index python-dotenv --upgrade

  Using cached ragas-0.1.20-py3-none-any.whl.metadata (5.5 kB)
  Using cached langchain_core-0.2.41-py3-none-any.whl.metadata (6.2 kB)
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain-0.2.16-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_text_splitters-0.2.4-py3-none-any.whl.metadata (2.3 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.2.17-py3-none-any.whl.metadata (2.7 kB)
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_openai-0.1.25-py3-none-any.whl.metadata (2.6 kB)
Using cached ragas-0.1.20-py3-none-any.whl (190 kB)
Using cached langchain_core-0.2.41-py3-none-any.whl (397 kB

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.1.0 requires langchain-core<0.4,>=0.3.0, but you have langchain-core 0.2.41 which is incompatible.


In [13]:
%pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


Repo card metadata block was not found. Setting CardData to empty.


In [15]:
amnesty_qa[0]

{'question': 'What are the global implications of the USA Supreme Court ruling on abortion?',
 'ground_truth': "The global implications of the USA Supreme Court ruling on abortion are significant. The ruling has led to limited or no access to abortion for one in three women and girls of reproductive age in states where abortion access is restricted. These states also have weaker maternal health support, higher maternal death rates, and higher child poverty rates. Additionally, the ruling has had an impact beyond national borders due to the USA's geopolitical and cultural influence globally. Organizations and activists worldwide are concerned that the ruling may inspire anti-abortion legislative and policy attacks in other countries. The ruling has also hindered progressive law reform and the implementation of abortion guidelines in certain African countries. Furthermore, the ruling has created a chilling effect in international policy spaces, empowering anti-abortion actors to undermin

d:\workspace\arxplore2\.venv_eval\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
from datasets import load_dataset

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2", trust_remote_code=True)["eval"]


from langchain_openai.chat_models import ChatOpenAI
# from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings


# import metrics
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
# from ragas.metrics.critique import SUPPORTED_ASPECTS, harmfulness

# metrics you chose
metrics = [faithfulness, answer_relevancy, context_precision, context_recall]

from ragas.run_config import RunConfig
from ragas.metrics.base import MetricWithLLM, MetricWithEmbeddings


# util function to init Ragas Metrics
def init_ragas_metrics(metrics, llm, embedding):
    for metric in metrics:
        if isinstance(metric, MetricWithLLM):
            metric.llm = llm
        if isinstance(metric, MetricWithEmbeddings):
            metric.embeddings = embedding
        run_config = RunConfig()
        metric.init(run_config)
# wrappers
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

llm = ChatOpenAI(openai_api_base="http://localhost:5000/v1", openai_api_key="lm-studio")
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/allenai-specter",
                                 model_kwargs={"device": "cpu"})

init_ragas_metrics(
    metrics,
    llm=LangchainLLMWrapper(llm),
    embedding=LangchainEmbeddingsWrapper(emb),
)

from ragas import evaluate
# from ragas.run_config import RunConfig

# run_config = RunConfig(max_workers=0)
# sample = {
#     "question": ["What is ChatGPT?", "What is the purpose of ChatGPT?"],
#     "contexts": [["ChatGPT is a conversational AI model developed by OpenAI."], ["ChatGPT is a conversational AI model developed by OpenAI."]],
#     "ground_thruth": ["ChatGPT is a conversational AI model developed by OpenAI.", "ChatGPT is a conversational AI model developed by OpenAI."],
#     "response": ["ChatGPT is a conversational AI model developed by OpenAI.", "ChatGPT is a conversational AI model developed by OpenAI."],
# }

# import datasets

# custom_Ds = datasets.Dataset.from_dict(sample)
result = evaluate(
    # custom_Ds,
    amnesty_qa,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    # run_config=run_config,
)

result

Evaluating:   5%|▌         | 4/80 [02:58<40:37, 32.07s/it]   Exception raised in Job[15]: TimeoutError()
Exception raised in Job[13]: TimeoutError()
Exception raised in Job[48]: TimeoutError()
Exception raised in Job[50]: TimeoutError()
Exception raised in Job[17]: TimeoutError()
Exception raised in Job[46]: TimeoutError()
Exception raised in Job[14]: TimeoutError()
Exception raised in Job[12]: TimeoutError()
Exception raised in Job[16]: TimeoutError()
Exception raised in Job[49]: TimeoutError()
Exception raised in Job[53]: TimeoutError()
Exception raised in Job[52]: TimeoutError()
Evaluating:  24%|██▍       | 19/80 [05:01<14:54, 14.67s/it]Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Exception raised in Job[27]: TimeoutError()
Exception raised in Job[60]: TimeoutError()
Exception raised in Job[56]: TimeoutError()
Exception raised in Job[58]: TimeoutError()
Exception raised in Job[57]: TimeoutError()
Exception ra

{'context_precision': nan, 'faithfulness': 1.0000, 'answer_relevancy': 0.9877, 'context_recall': 0.6000}